In [1]:
import sys

sys.path.append("../")
from calculus_path_mod.term_engine import *
from calculus_path_mod.reduction_strategy import *

Lambda = Abstraction
App = Application

# <span style="color: #A1F99C">TEST PROCEDURE</span>

In [2]:
rs = RandomInnerStrategy(prob_norm="softmax")


def test_procedure(term, reduction_strategy=rs, print_steps=True,
                   steps_lim=60, assert_result=""):
    term = term._update_bound_vars()
    n_steps = 0
    is_term_normalized = True
    if print_steps:
        print(f"{n_steps}: {term}")
    while term.redexes:
        term = term._beta_conversion(reduction_strategy)
        term = term._update_bound_vars()
        n_steps += 1
        if print_steps:
            print(f"{n_steps}: {term}")
        if n_steps > steps_lim:
            is_term_normalized = False
            break

    print(f"\n\nreduction steps: {n_steps}")
    print(f"norm term:       {term}" if is_term_normalized else f"NOT norm term: {term}")

# <span style="color: #A1F99C">USEFUL TERMS</span>

In [3]:
def ite_term():
    x, y, c = Var(), Var(), Var()
    x_, y_, c_ = Atom(x), Atom(y), Atom(c)
    return Lambda(c, Lambda(x, Lambda(y, multi_app_term(c_, x_, y_))))


def true_term():
    x, y = Var(), Var()
    x_ = Atom(x)
    return Lambda(x, Lambda(y, x_))


def num_zero_term():
    s, z = Var(), Var()
    z_ = Atom(z)
    return Lambda(s, Lambda(z, z_))


def num_term(n: int):
    if n <= 0:
        return num_zero_term()
    s, z = Var(), Var()
    s_, z_ = Atom(s), Atom(z)
    core_term = App(s_, z_)
    for _ in range(n - 1):
        core_term = App(s_, core_term)
    return Lambda(s, Lambda(z, core_term))


def multi_app_term(term_0: Term, term_1: Term, *terms: Term):
    res_app_term = App(term_0, term_1)
    for term in terms:
        res_app_term = App(res_app_term, term)
    return res_app_term


def false_term():
    x, y = Var(), Var()
    y_ = Atom(y)
    return Lambda(x, Lambda(y, y_))


def pair_term():
    x, y, p = Var(), Var(), Var()
    x_, y_, p_ = Atom(x), Atom(y), Atom(p)
    return Lambda(x, Lambda(y, Lambda(p, multi_app_term(p_, x_, y_))))


def first_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, true_term()))


def second_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, false_term()))


def succ_term():
    x, y, n = Var(), Var(), Var()
    x_, y_, n_ = Atom(x), Atom(y), Atom(n)
    return Lambda(n, Lambda(x, Lambda(y, App(x_, multi_app_term(n_, x_, y_)))))


def sinc_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(
        p,
        multi_app_term(
            pair_term(),
            App(second_term(), p_),
            App(succ_term(), App(second_term(), p_)),
        ),
    )


def pred_term():
    n, f, x, g, h, u = Var(), Var(), Var(), Var(), Var(), Var()
    n_, f_, x_ = Atom(n), Atom(f), Atom(x)
    g_, h_, u_ = Atom(g), Atom(h), Atom(u)

    return Lambda(n, Lambda(f, Lambda(x, multi_app_term(
        n_,
        Lambda(g, Lambda(h, App(h_, App(g_, f_)))),
        Lambda(u, x_),
        Lambda(u, u_)
    ))))


def plus_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(m, Lambda(n, multi_app_term(n_, succ_term(), m_)))


def subtract_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(n, Lambda(m, multi_app_term(m_, pred_term(), n_)))


def iszero_term():
    x, n = Var(), Var()
    n_ = Atom(n)
    return Lambda(n, App(App(n_, Lambda(x, false_term())), true_term()))


def leq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n, Lambda(m, App(iszero_term(), multi_app_term(subtract_term(), n_, m_)))
    )


def and_term():
    a, b = Var(), Var()
    a_, b_ = Atom(a), Atom(b)
    return Lambda(a, Lambda(b, multi_app_term(ite_term(), a_, b_, a_)))


def eq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n,
        Lambda(
            m,
            multi_app_term(
                and_term(),
                multi_app_term(leq_term(), n_, m_),
                multi_app_term(leq_term(), m_, n_),
            ),
        ),
    )

# <span style="color: #A1F99C">1.</span>
<span style="color:#bb91f3">(λx. (λy.y)) ((λx. (xx)) (λx. (xx)))</span> == <span style="color:#f1bd47">I</span> == <span style="color:#f1bd47">λx. x</span>
<span style="color:#c9fafa">This term reduce in one step by LO, RO strategy
And reduce in the same terms with other strategy.</span>

In [4]:
def term_test_1():
    x, y = Var(), Var()
    x_, y_ = Atom(x), Atom(y)

    return App(
        Lambda(x, Lambda(y, y_)),
        App(
            Lambda(x, App(x_, x_)),
            Lambda(x, App(x_, x_))
        )
    )


test_procedure(term_test_1())

0: ((λ#2. (λ#3. #3)) ((λ#4. (#4 #4)) (λ#5. (#5 #5))))
1: (λ#6. #6)


reduction steps: 1
norm term:       (λ#6. #6)


# <span style="color: #A1F99C">2.</span>
<span style="color:#bb91f3">(ITE TRUE 3 1)</span> == <span style="color:#f1bd47>">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [5]:
def term_test_2():
    return multi_app_term(ite_term(), true_term(), num_term(3), num_term(1))


test_procedure(term_test_2())

0: ((((λ#16. (λ#17. (λ#18. ((#16 #17) #18)))) (λ#19. (λ#20. #19))) (λ#21. (λ#22. (#21 (#21 (#21 #22)))))) (λ#23. (λ#24. (#23 #24))))
1: (((λ#25. (λ#26. (((λ#27. (λ#28. #27)) #25) #26))) (λ#29. (λ#30. (#29 (#29 (#29 #30)))))) (λ#31. (λ#32. (#31 #32))))
2: (((λ#33. (λ#34. ((λ#35. #33) #34))) (λ#36. (λ#37. (#36 (#36 (#36 #37)))))) (λ#38. (λ#39. (#38 #39))))
3: (((λ#40. (λ#41. #40)) (λ#42. (λ#43. (#42 (#42 (#42 #43)))))) (λ#44. (λ#45. (#44 #45))))
4: ((λ#46. (λ#47. (λ#48. (#47 (#47 (#47 #48)))))) (λ#49. (λ#50. (#49 #50))))
5: (λ#51. (λ#52. (#51 (#51 (#51 #52)))))


reduction steps: 5
norm term:       (λ#51. (λ#52. (#51 (#51 (#51 #52)))))


# <span style="color: #A1F99C">3.</span>
<span style="color:#bb91f3">(ITE FALSE 0 4)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [6]:
def term_test_3():
    return multi_app_term(ite_term(), false_term(), num_term(0), num_term(4))


test_procedure(term_test_3())

0: ((((λ#62. (λ#63. (λ#64. ((#62 #63) #64)))) (λ#65. (λ#66. #66))) (λ#67. (λ#68. #68))) (λ#69. (λ#70. (#69 (#69 (#69 (#69 #70)))))))
1: (((λ#71. (λ#72. (((λ#73. (λ#74. #74)) #71) #72))) (λ#75. (λ#76. #76))) (λ#77. (λ#78. (#77 (#77 (#77 (#77 #78)))))))
2: (((λ#79. (λ#80. ((λ#81. #81) #80))) (λ#82. (λ#83. #83))) (λ#84. (λ#85. (#84 (#84 (#84 (#84 #85)))))))
3: (((λ#86. (λ#87. #87)) (λ#88. (λ#89. #89))) (λ#90. (λ#91. (#90 (#90 (#90 (#90 #91)))))))
4: ((λ#92. #92) (λ#93. (λ#94. (#93 (#93 (#93 (#93 #94)))))))
5: (λ#95. (λ#96. (#95 (#95 (#95 (#95 #96))))))


reduction steps: 5
norm term:       (λ#95. (λ#96. (#95 (#95 (#95 (#95 #96))))))


# <span style="color: #A1F99C">4.</span>
<span style="color:#bb91f3">(PRED 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [7]:
def term_test_4():
    return multi_app_term(pred_term(), num_term(0))


test_procedure(term_test_4())

0: ((λ#105. (λ#106. (λ#107. (((#105 (λ#108. (λ#109. (#109 (#108 #106))))) (λ#110. #107)) (λ#111. #111))))) (λ#112. (λ#113. #113)))
1: (λ#114. (λ#115. ((((λ#116. (λ#117. #117)) (λ#118. (λ#119. (#119 (#118 #114))))) (λ#120. #115)) (λ#121. #121))))
2: (λ#122. (λ#123. (((λ#124. #124) (λ#125. #123)) (λ#126. #126))))
3: (λ#127. (λ#128. ((λ#129. #128) (λ#130. #130))))
4: (λ#131. (λ#132. #132))


reduction steps: 4
norm term:       (λ#131. (λ#132. #132))


# <span style="color: #A1F99C">5.</span>
<span style="color:#bb91f3">(PRED 5)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [8]:
def term_test_5():
    return multi_app_term(pred_term(), num_term(5))


test_procedure(term_test_5())

0: ((λ#141. (λ#142. (λ#143. (((#141 (λ#144. (λ#145. (#145 (#144 #142))))) (λ#146. #143)) (λ#147. #147))))) (λ#148. (λ#149. (#148 (#148 (#148 (#148 (#148 #149))))))))
1: (λ#150. (λ#151. ((((λ#152. (λ#153. (#152 (#152 (#152 (#152 (#152 #153))))))) (λ#154. (λ#155. (#155 (#154 #150))))) (λ#156. #151)) (λ#157. #157))))
2: (λ#158. (λ#159. (((λ#160. ((λ#161. (λ#162. (#162 (#161 #158)))) ((λ#163. (λ#164. (#164 (#163 #158)))) ((λ#165. (λ#166. (#166 (#165 #158)))) ((λ#167. (λ#168. (#168 (#167 #158)))) ((λ#169. (λ#170. (#170 (#169 #158)))) #160)))))) (λ#171. #159)) (λ#172. #172))))
3: (λ#173. (λ#174. (((λ#175. ((λ#176. (λ#177. (#177 (#176 #173)))) ((λ#178. (λ#179. (#179 (#178 #173)))) ((λ#180. (λ#181. (#181 (#180 #173)))) ((λ#182. (λ#183. (#183 (#182 #173)))) (λ#184. (#184 (#175 #173)))))))) (λ#185. #174)) (λ#186. #186))))
4: (λ#187. (λ#188. (((λ#189. ((λ#190. (λ#191. (#191 (#190 #187)))) ((λ#192. (λ#193. (#193 (#192 #187)))) ((λ#194. (λ#195. (#195 (#194 #187)))) (λ#196. (#196 ((λ#197. (#197 (#18

# <span style="color: #A1F99C">6.</span>
<span style="color:#bb91f3">(PLUS 0 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [9]:
def term_test_6():
    return multi_app_term(plus_term(), num_term(0), num_term(0))


test_procedure(term_test_6())

0: (((λ#286. (λ#287. ((#287 (λ#288. (λ#289. (λ#290. (#289 ((#288 #289) #290)))))) #286))) (λ#291. (λ#292. #292))) (λ#293. (λ#294. #294)))
1: ((λ#295. ((#295 (λ#296. (λ#297. (λ#298. (#297 ((#296 #297) #298)))))) (λ#299. (λ#300. #300)))) (λ#301. (λ#302. #302)))
2: (((λ#303. (λ#304. #304)) (λ#305. (λ#306. (λ#307. (#306 ((#305 #306) #307)))))) (λ#308. (λ#309. #309)))
3: ((λ#310. #310) (λ#311. (λ#312. #312)))
4: (λ#313. (λ#314. #314))


reduction steps: 4
norm term:       (λ#313. (λ#314. #314))


# <span style="color: #A1F99C">7.</span>
<span style="color:#bb91f3">(PLUS 2 3)</span> == <span style="color:#f1bd47">5</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x (x y)))))</span>

In [10]:
def term_test_7():
    return multi_app_term(plus_term(), num_term(2), num_term(3))


test_procedure(term_test_7())

0: (((λ#324. (λ#325. ((#325 (λ#326. (λ#327. (λ#328. (#327 ((#326 #327) #328)))))) #324))) (λ#329. (λ#330. (#329 (#329 #330))))) (λ#331. (λ#332. (#331 (#331 (#331 #332))))))
1: ((λ#333. ((#333 (λ#334. (λ#335. (λ#336. (#335 ((#334 #335) #336)))))) (λ#337. (λ#338. (#337 (#337 #338)))))) (λ#339. (λ#340. (#339 (#339 (#339 #340))))))
2: (((λ#341. (λ#342. (#341 (#341 (#341 #342))))) (λ#343. (λ#344. (λ#345. (#344 ((#343 #344) #345)))))) (λ#346. (λ#347. (#346 (#346 #347)))))
3: ((λ#348. ((λ#349. (λ#350. (λ#351. (#350 ((#349 #350) #351))))) ((λ#352. (λ#353. (λ#354. (#353 ((#352 #353) #354))))) ((λ#355. (λ#356. (λ#357. (#356 ((#355 #356) #357))))) #348)))) (λ#358. (λ#359. (#358 (#358 #359)))))
4: ((λ#360. ((λ#361. (λ#362. (λ#363. (#362 ((#361 #362) #363))))) ((λ#364. (λ#365. (λ#366. (#365 ((#364 #365) #366))))) (λ#367. (λ#368. (#367 ((#360 #367) #368))))))) (λ#369. (λ#370. (#369 (#369 #370)))))
5: ((λ#371. ((λ#372. (λ#373. (λ#374. (#373 ((#372 #373) #374))))) (λ#375. (λ#376. (#375 (((λ#377. (λ#37

# <span style="color: #A1F99C">8.</span>
<span style="color:#bb91f3">(SUBTRACT 4 1)</span> == <span style="color:#f1bd47">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [11]:
def term_test_8():
    return multi_app_term(subtract_term(), num_term(4), num_term(1))


test_procedure(term_test_8())

0: (((λ#437. (λ#438. ((#438 (λ#439. (λ#440. (λ#441. (((#439 (λ#442. (λ#443. (#443 (#442 #440))))) (λ#444. #441)) (λ#445. #445)))))) #437))) (λ#446. (λ#447. (#446 (#446 (#446 (#446 #447))))))) (λ#448. (λ#449. (#448 #449))))
1: ((λ#450. ((#450 (λ#451. (λ#452. (λ#453. (((#451 (λ#454. (λ#455. (#455 (#454 #452))))) (λ#456. #453)) (λ#457. #457)))))) (λ#458. (λ#459. (#458 (#458 (#458 (#458 #459)))))))) (λ#460. (λ#461. (#460 #461))))
2: (((λ#462. (λ#463. (#462 #463))) (λ#464. (λ#465. (λ#466. (((#464 (λ#467. (λ#468. (#468 (#467 #465))))) (λ#469. #466)) (λ#470. #470)))))) (λ#471. (λ#472. (#471 (#471 (#471 (#471 #472)))))))
3: ((λ#473. ((λ#474. (λ#475. (λ#476. (((#474 (λ#477. (λ#478. (#478 (#477 #475))))) (λ#479. #476)) (λ#480. #480))))) #473)) (λ#481. (λ#482. (#481 (#481 (#481 (#481 #482)))))))
4: ((λ#483. (λ#484. (λ#485. (((#483 (λ#486. (λ#487. (#487 (#486 #484))))) (λ#488. #485)) (λ#489. #489))))) (λ#490. (λ#491. (#490 (#490 (#490 (#490 #491)))))))
5: (λ#492. (λ#493. ((((λ#494. (λ#495. (#494 (

# <span style="color: #A1F99C">9.</span>
<span style="color:#bb91f3">(SUBTRACT 3 5)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [12]:
def term_test_9():
    return multi_app_term(subtract_term(), num_term(3), num_term(5))


test_procedure(term_test_9())

0: (((λ#602. (λ#603. ((#603 (λ#604. (λ#605. (λ#606. (((#604 (λ#607. (λ#608. (#608 (#607 #605))))) (λ#609. #606)) (λ#610. #610)))))) #602))) (λ#611. (λ#612. (#611 (#611 (#611 #612)))))) (λ#613. (λ#614. (#613 (#613 (#613 (#613 (#613 #614))))))))
1: ((λ#615. ((#615 (λ#616. (λ#617. (λ#618. (((#616 (λ#619. (λ#620. (#620 (#619 #617))))) (λ#621. #618)) (λ#622. #622)))))) (λ#623. (λ#624. (#623 (#623 (#623 #624))))))) (λ#625. (λ#626. (#625 (#625 (#625 (#625 (#625 #626))))))))
2: (((λ#627. (λ#628. (#627 (#627 (#627 (#627 (#627 #628))))))) (λ#629. (λ#630. (λ#631. (((#629 (λ#632. (λ#633. (#633 (#632 #630))))) (λ#634. #631)) (λ#635. #635)))))) (λ#636. (λ#637. (#636 (#636 (#636 #637))))))
3: ((λ#638. ((λ#639. (λ#640. (λ#641. (((#639 (λ#642. (λ#643. (#643 (#642 #640))))) (λ#644. #641)) (λ#645. #645))))) ((λ#646. (λ#647. (λ#648. (((#646 (λ#649. (λ#650. (#650 (#649 #647))))) (λ#651. #648)) (λ#652. #652))))) ((λ#653. (λ#654. (λ#655. (((#653 (λ#656. (λ#657. (#657 (#656 #654))))) (λ#658. #655)) (λ#659. #6

# <span style="color: #A1F99C">10.</span>
<span style="color:#bb91f3">(LEQ 3 2)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [13]:
def term_test_10():
    return multi_app_term(leq_term(), num_term(3), num_term(2))


test_procedure(term_test_10())

0: (((λ#1611. (λ#1612. ((λ#1613. ((#1613 (λ#1614. (λ#1615. (λ#1616. #1616)))) (λ#1617. (λ#1618. #1617)))) (((λ#1619. (λ#1620. ((#1620 (λ#1621. (λ#1622. (λ#1623. (((#1621 (λ#1624. (λ#1625. (#1625 (#1624 #1622))))) (λ#1626. #1623)) (λ#1627. #1627)))))) #1619))) #1611) #1612)))) (λ#1628. (λ#1629. (#1628 (#1628 (#1628 #1629)))))) (λ#1630. (λ#1631. (#1630 (#1630 #1631)))))
1: (((λ#1632. (λ#1633. ((λ#1634. ((#1634 (λ#1635. (λ#1636. (λ#1637. #1637)))) (λ#1638. (λ#1639. #1638)))) ((λ#1640. ((#1640 (λ#1641. (λ#1642. (λ#1643. (((#1641 (λ#1644. (λ#1645. (#1645 (#1644 #1642))))) (λ#1646. #1643)) (λ#1647. #1647)))))) #1632)) #1633)))) (λ#1648. (λ#1649. (#1648 (#1648 (#1648 #1649)))))) (λ#1650. (λ#1651. (#1650 (#1650 #1651)))))
2: (((λ#1652. (λ#1653. ((((λ#1654. ((#1654 (λ#1655. (λ#1656. (λ#1657. (((#1655 (λ#1658. (λ#1659. (#1659 (#1658 #1656))))) (λ#1660. #1657)) (λ#1661. #1661)))))) #1652)) #1653) (λ#1662. (λ#1663. (λ#1664. #1664)))) (λ#1665. (λ#1666. #1665))))) (λ#1667. (λ#1668. (#1667 (#1667 (#1

# <span style="color: #A1F99C">11.</span>
<span style="color:#bb91f3">(LEQ 2 5)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [14]:
def term_test_11():
    return multi_app_term(leq_term(), num_term(2), num_term(5))


test_procedure(term_test_11())

0: (((λ#2100. (λ#2101. ((λ#2102. ((#2102 (λ#2103. (λ#2104. (λ#2105. #2105)))) (λ#2106. (λ#2107. #2106)))) (((λ#2108. (λ#2109. ((#2109 (λ#2110. (λ#2111. (λ#2112. (((#2110 (λ#2113. (λ#2114. (#2114 (#2113 #2111))))) (λ#2115. #2112)) (λ#2116. #2116)))))) #2108))) #2100) #2101)))) (λ#2117. (λ#2118. (#2117 (#2117 #2118))))) (λ#2119. (λ#2120. (#2119 (#2119 (#2119 (#2119 (#2119 #2120))))))))
1: (((λ#2121. (λ#2122. ((λ#2123. ((#2123 (λ#2124. (λ#2125. (λ#2126. #2126)))) (λ#2127. (λ#2128. #2127)))) ((λ#2129. ((#2129 (λ#2130. (λ#2131. (λ#2132. (((#2130 (λ#2133. (λ#2134. (#2134 (#2133 #2131))))) (λ#2135. #2132)) (λ#2136. #2136)))))) #2121)) #2122)))) (λ#2137. (λ#2138. (#2137 (#2137 #2138))))) (λ#2139. (λ#2140. (#2139 (#2139 (#2139 (#2139 (#2139 #2140))))))))
2: (((λ#2141. (λ#2142. ((λ#2143. ((#2143 (λ#2144. (λ#2145. (λ#2146. #2146)))) (λ#2147. (λ#2148. #2147)))) ((#2142 (λ#2149. (λ#2150. (λ#2151. (((#2149 (λ#2152. (λ#2153. (#2153 (#2152 #2150))))) (λ#2154. #2151)) (λ#2155. #2155)))))) #2141)))) (λ#

# <span style="color: #A1F99C">12.</span>
<span style="color:#bb91f3">(EQ 2 2)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [15]:
def term_test_12():
    return multi_app_term(eq_term(), num_term(2), num_term(2))


test_procedure(term_test_12(), steps_lim=100)

0: (((λ#3138. (λ#3139. (((λ#3140. (λ#3141. ((((λ#3142. (λ#3143. (λ#3144. ((#3142 #3143) #3144)))) #3140) #3141) #3140))) (((λ#3145. (λ#3146. ((λ#3147. ((#3147 (λ#3148. (λ#3149. (λ#3150. #3150)))) (λ#3151. (λ#3152. #3151)))) (((λ#3153. (λ#3154. ((#3154 (λ#3155. (λ#3156. (λ#3157. (((#3155 (λ#3158. (λ#3159. (#3159 (#3158 #3156))))) (λ#3160. #3157)) (λ#3161. #3161)))))) #3153))) #3145) #3146)))) #3138) #3139)) (((λ#3162. (λ#3163. ((λ#3164. ((#3164 (λ#3165. (λ#3166. (λ#3167. #3167)))) (λ#3168. (λ#3169. #3168)))) (((λ#3170. (λ#3171. ((#3171 (λ#3172. (λ#3173. (λ#3174. (((#3172 (λ#3175. (λ#3176. (#3176 (#3175 #3173))))) (λ#3177. #3174)) (λ#3178. #3178)))))) #3170))) #3162) #3163)))) #3139) #3138)))) (λ#3179. (λ#3180. (#3179 (#3179 #3180))))) (λ#3181. (λ#3182. (#3181 (#3181 #3182)))))
1: (((λ#3183. (λ#3184. (((λ#3185. (λ#3186. ((((λ#3187. (λ#3188. (λ#3189. ((#3187 #3188) #3189)))) #3185) #3186) #3185))) (((λ#3190. (λ#3191. ((λ#3192. ((#3192 (λ#3193. (λ#3194. (λ#3195. #3195)))) (λ#3196. (λ#3197.

# <span style="color: #A1F99C">13.</span>
<span style="color:#bb91f3">(EQ 1 4)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [16]:
def term_test_13():
    return multi_app_term(eq_term(), num_term(1), num_term(4))


test_procedure(term_test_13(), steps_lim=100)

0: (((λ#6256. (λ#6257. (((λ#6258. (λ#6259. ((((λ#6260. (λ#6261. (λ#6262. ((#6260 #6261) #6262)))) #6258) #6259) #6258))) (((λ#6263. (λ#6264. ((λ#6265. ((#6265 (λ#6266. (λ#6267. (λ#6268. #6268)))) (λ#6269. (λ#6270. #6269)))) (((λ#6271. (λ#6272. ((#6272 (λ#6273. (λ#6274. (λ#6275. (((#6273 (λ#6276. (λ#6277. (#6277 (#6276 #6274))))) (λ#6278. #6275)) (λ#6279. #6279)))))) #6271))) #6263) #6264)))) #6256) #6257)) (((λ#6280. (λ#6281. ((λ#6282. ((#6282 (λ#6283. (λ#6284. (λ#6285. #6285)))) (λ#6286. (λ#6287. #6286)))) (((λ#6288. (λ#6289. ((#6289 (λ#6290. (λ#6291. (λ#6292. (((#6290 (λ#6293. (λ#6294. (#6294 (#6293 #6291))))) (λ#6295. #6292)) (λ#6296. #6296)))))) #6288))) #6280) #6281)))) #6257) #6256)))) (λ#6297. (λ#6298. (#6297 #6298)))) (λ#6299. (λ#6300. (#6299 (#6299 (#6299 (#6299 #6300)))))))
1: (((λ#6301. (λ#6302. (((λ#6303. (λ#6304. (((λ#6305. (λ#6306. ((#6303 #6305) #6306))) #6304) #6303))) (((λ#6307. (λ#6308. ((λ#6309. ((#6309 (λ#6310. (λ#6311. (λ#6312. #6312)))) (λ#6313. (λ#6314. #6313))))